In [1]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Mode
input_shape = (224, 224, 1
num_classes = 
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('data', target_size=input_shape[:2], batch_size=32, color_mode='grayscale', class_mode='categorical')
test_generator = test_datagen.flow_from_directory('data', target_size=input_shape[:2], batch_size=32, color_mode='grayscale', class_mode='categorical'
inputs = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(inputs)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=10, validation_data=test_generator)

model.save('face_mask_detector_model.h5')

Found 1376 images belonging to 2 classes.
Found 1376 images belonging to 2 classes.
Epoch 1/10
43/43 [==============================] - 86s 2s/step - loss: 0.7758 - accuracy: 0.5625 - val_loss: 0.5892 - val_accuracy: 0.6555
Epoch 2/10
43/43 [==============================] - 88s 2s/step - loss: 0.4527 - accuracy: 0.7914 - val_loss: 0.2981 - val_accuracy: 0.8728
Epoch 3/10
43/43 [==============================] - 88s 2s/step - loss: 0.2328 - accuracy: 0.9092 - val_loss: 0.2618 - val_accuracy: 0.8859
Epoch 4/10
43/43 [==============================] - 88s 2s/step - loss: 0.1705 - accuracy: 0.9346 - val_loss: 0.1919 - val_accuracy: 0.9208
Epoch 5/10
43/43 [==============================] - 87s 2s/step - loss: 0.1460 - accuracy: 0.9440 - val_loss: 0.0719 - val_accuracy: 0.9753
Epoch 6/10
43/43 [==============================] - 87s 2s/step - loss: 0.1157 - accuracy: 0.9578 - val_loss: 0.0520 - val_accuracy: 0.9833
Epoch 7/10
43/43 [==============================] - 88s 2s/step - loss: 0.18

In [2]:

model = load_model('face_mask_detector_model.h5')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)

    for (x, y, w, h) in faces:

        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (224, 224))
        roi_gray = roi_gray.astype('float') / 255.0
        roi_gray = np.expand_dims(roi_gray, axis=0)
        roi_gray = np.expand_dims(roi_gray, axis=-1)


        mask = model.predict(roi_gray)[0][0]
        if mask > 0.5:
            label = 'Mask'
            color = (0, 255, 0)
        else:
            label = 'No Mask'
            color = (0, 0, 255)


        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)

    cv2.imshow('Face Mask Detector', frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 34ms/step
